In [1]:
pip install pandas lxml geopandas fiona

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import geopandas as gpd
from lxml import etree
from zipfile import ZipFile
import os
import shutil

# Function to parse HTML and extract key/value pairs
def parse_html(description):
    html = etree.HTML(description)
    rows = html.xpath('//table//tr')
    data = {}
    for row in rows:
        cells = row.xpath('./td/text()')
        if len(cells) == 2:
            data[cells[0].strip()] = cells[1].strip()
    return data

# Unzip KMZ to KML
def kmz_to_kml(kmz_file, output_path):
    with ZipFile(kmz_file, 'r') as zip_ref:
        zip_ref.extractall(output_path)
        kml_file = [f for f in os.listdir(output_path) if f.endswith('.kml')][0]
        return os.path.join(output_path, kml_file)

# Convert KML to GeoDataFrame and extract description
def kml_to_gdf(kml_file):
    gdf = gpd.read_file(kml_file, driver='KML')
    gdf['attributes'] = gdf['Description'].apply(parse_html)
    for key in set.union(*[set(d.keys()) for d in gdf['attributes'] if isinstance(d, dict)]):
        gdf[key] = gdf['attributes'].apply(lambda d: d.get(key, None))
    return gdf.drop(columns=['Description', 'attributes'])

# Main conversion function
def convert_kmz_to_shp(kmz_file, shapefile_path):
    temp_dir = 'temp_kml'
    os.makedirs(temp_dir, exist_ok=True)
    
    try:
        # Convert KMZ to KML
        kml_file = kmz_to_kml(kmz_file, temp_dir)
        # Convert KML to GeoDataFrame
        gdf = kml_to_gdf(kml_file)
        # Save to Shapefile
        gdf.to_file(shapefile_path, driver='ESRI Shapefile')
        print(f"Shapefile has been saved to {shapefile_path}")
    finally:
        # Cleanup temporary files
        shutil.rmtree(temp_dir)

# Example usage
kmz_path = 'BLOS_BlueARedF.kmz'
output_shp_path = 'TNbikeroutesclean'
convert_kmz_to_shp(kmz_path, output_shp_path)

Shapefile has been saved to TNbikeroutesclean
